<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [66]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup

In [67]:
# настроки к базе
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [68]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [69]:
vacancies_query = f'''select count(*)
           from VACANCIES
        '''
vacancies_df = pd.read_sql_query(vacancies_query, connection)

In [70]:
# результат запроса
vacancies_df

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [71]:
eployers_query = f'''select count(*)
           from EMPLOYERS
        '''
eployers_df = pd.read_sql_query(eployers_query, connection)

In [72]:
eployers_df

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [73]:
areas_query = f'''select count(*)
           from AREAS
        '''
areas_df = pd.read_sql_query(areas_query, connection)

In [74]:
areas_df

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [75]:
industries_query = f'''select count(*)
           from INDUSTRIES
        '''
industries_df = pd.read_sql_query(industries_query, connection)

In [76]:
industries_df

,count
0,294


***

# Вывод
Вакансий больше чем служащих, значит некоторые служащие могут иметь больше одной одной вакансии

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [77]:
query_0 = f'''
select a.name, count(v.id) as cnt
    from  EMPLOYERS e, AREAS a, VACANCIES v
    where v.area_id = a.id and v.employer_id = e.id
    group by a.id
    order by cnt desc
        '''
df_0 = pd.read_sql_query(query_0, connection)

In [78]:
df_0.head()

,name,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [79]:
query_1 = f'''
select count(*) from  VACANCIES where salary_from is not null or salary_to is not null
        '''
df_1 = pd.read_sql_query(query_1, connection)

In [80]:
df_1

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [81]:
query_2 = f'''
     select avg(salary_from) as salary_mean_min, avg(salary_to) as salary_mean_max from  VACANCIES
        '''
df_2 = pd.read_sql_query(query_2, connection)

In [82]:
df_2

,salary_mean_min,salary_mean_max
0,71064.657901,110536.741923


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [83]:
query_3 = f'''
     select v.schedule, v.employment, count(*) as cnt from  public.VACANCIES v group by v.schedule, v.employment order by cnt desc
        '''
df_3 = pd.read_sql_query(query_3, connection)

In [84]:
df_3

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [85]:
query_4 = f'''
     select v.experience as "Требуемый опыт работы", count(*) as cnt from  VACANCIES v group by v.experience order by cnt
        '''
df_4 = pd.read_sql_query(query_4, connection)

In [86]:
df_4

,Требуемый опыт работы,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

# Выводы
- Работадатели имеют одну и более вакансий, возможно эти вакансии в разных сферах или разную позицию по опыту
- Требования по зарплате в среднем находится от 71 до 110 тыс рублей
- Наиболее количество вакансий требует опыта от 1 до 3 лет
- Наибольшее количество вакансий в крупных городах (миллионниках) таких как Минск, Москва


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [87]:
query_5 = f'''
     select e.name, count(*) cnt from  VACANCIES v, EMPLOYERS e where v.employer_id = e.id group by e.name order by cnt desc
        '''
df_5 = pd.read_sql_query(query_5, connection)

In [88]:
df_5

,name,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
...,...,...
14761,им. Тьюринга,1
14762,LLC Novalab Tech,1
14763,Компания Мир упаковки,1
14764,Лифинцев Олег Иванович,1


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [89]:
query_6 = f'''
select
    a.name "Регион",
    count(distinct e.id) "Количество работодателей",
    count(distinct v.id) "Количество вакансий"
from public.areas a
    left join employers e
    ON a.id = e.area
    left join vacancies v
    ON a.id = v.area_id
where v.name is null
group by 1
order by 2 desc
        '''
df_6 = pd.read_sql_query(query_6, connection)

In [90]:
df_6.head()

,Регион,Количество работодателей,Количество вакансий
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [91]:
query_7 = f'''
select e.name "Работадатель", count(distinct v.area_id) "Количество вакансий в регионах"
    from VACANCIES v 
    left join EMPLOYERS e on e.id = v.employer_id
        group by e.name order by 2 desc
        '''
df_7 = pd.read_sql_query(query_7, connection)

In [92]:
df_7.head()

,Работадатель,Количество вакансий в регионах
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [93]:
query_8 = f'''
select count(distinct e.id) "Колличество работодателей у которых не указана сфера деятельности"
    from EMPLOYERS e
    left outer join EMPLOYERS_INDUSTRIES i on i.employer_id = e.id where i.industry_id is null
        '''
df_8 = pd.read_sql_query(query_8, connection)

In [94]:
df_8.head()

,Колличество работодателей у котор
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [95]:
query_9 = f'''
select e.name "Название компании"
    from EMPLOYERS e
    left join EMPLOYERS_INDUSTRIES i on i.employer_id = e.id group by e.name having count(i.industry_id) = 4 order by 1 
        '''
df_9 = pd.read_sql_query(query_9, connection)

In [96]:
df_9.head()

,Название компании
0,101 Интернет
1,21vek.by
2,2ГИС
3,2К
4,4 пикселя +


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [97]:
query_10 = f'''
select count(ei.employer_id) "Количество компаний"
  from EMPLOYERS_INDUSTRIES ei
  left join INDUSTRIES i on i.id = ei.industry_id where i.name = 'Разработка программного обеспечения'
        '''
df_10 = pd.read_sql_query(query_10, connection)

In [98]:
df_10.head()

,Количество компаний
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [99]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
links = page.find_all('a')
cities = list(filter(lambda city : city != '[a]', [link.text for link in links[46:63]]))
cities

['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Челябинск',
 'Красноярск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [100]:
query_11 = f'''
(
    select a.name, count(distinct v.id) as cnt
     from VACANCIES v
     left join EMPLOYERS e on e.id = v.employer_id
     left join AREAS a on a.id = v.area_id
             where a.name in {repr(tuple(map(str, cities)))} and e.name = 'Яндекс'
     group by a.name order by 2
 )
 union all
 (
     select 'Total' as Total, sum(t.cnt) from (
      select count(distinct v.id) as cnt
             from VACANCIES v
             left join EMPLOYERS e on e.id = v.employer_id
             left join AREAS a on a.id = v.area_id
             where a.name in {repr(tuple(map(str, cities)))} and e.name = 'Яндекс'
             group by a.name
     ) as t
     
)
'''
df_11 = pd.read_sql_query(query_11, connection)

In [101]:
df_11

,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

# Выводы по анализу работодателей
- Лидируеющие позиции по количеству вакансий занимаю самые крупные IT компании таки как Яндекс и т.д
- Так же самые крупные IT компании занимаю наиболее больший охват рабочих мест по регионам
- Среди всех вакансий присуствуют и те у которых не указана сфера деятельности

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [102]:
query_12 = f'''
select count(*) from public.VACANCIES where upper(name) LIKE upper('%data%') or upper(name) like upper('%данн%')
        '''
df_12 = pd.read_sql_query(query_12, connection)

In [103]:
df_12

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [104]:
query_13 = f'''
select count(*) from public.VACANCIES 
    where (
            upper(name) like upper('%data scientist%') 
            or upper(name) like upper('%data science%')
            or upper(name) like upper('%исследователь данных%')
            or (name LIKE '%ML%' and name not LIKE '%HTML%')
            or upper(name) like upper('%machine learning%')
            or upper(name) like upper('%машинн%обучен%')
        ) and (
            upper(name) like upper('%junior%')
            or experience = 'Нет опыта'
            or schedule = 'Стажировка'
        )
        '''
df_13 = pd.read_sql_query(query_13, connection)

In [105]:
df_13

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [106]:
query_14 = f'''
select count(*) from public.VACANCIES 
    where (
            upper(name) like upper('%data scientist%') 
            or upper(name) like upper('%data science%')
            or upper(name) like upper('%исследователь данных%')
            or (name LIKE '%ML%' and name not LIKE '%HTML%')
            or upper(name) like upper('%machine learning%')
            or upper(name) like upper('%машинн%обучен%')
        ) and (
           upper(key_skills) like upper('%SQL%')
           or upper(key_skills) like upper('%postgres%')
        )
'''
df_14 = pd.read_sql_query(query_14, connection)

In [107]:
df_14

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [108]:
query_15 = f'''
select count(*) from public.VACANCIES 
    where (
            upper(name) like upper('%data scientist%') 
            or upper(name) like upper('%data science%')
            or upper(name) like upper('%исследователь данных%')
            or (name LIKE '%ML%' and name not LIKE '%HTML%')
            or upper(name) like upper('%machine learning%')
            or upper(name) like upper('%машинн%обучен%')
        ) and (
           upper(key_skills) like upper('%Python%')
        )
'''
df_15 = pd.read_sql_query(query_15, connection)

In [109]:
df_15

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [110]:
query_16 = f'''
select avg(length(key_skills) - length(replace(key_skills, chr(9), '')) +1)
    from public.VACANCIES 
    where (
            upper(name) like upper('%data scientist%') 
            or upper(name) like upper('%data science%')
            or upper(name) like upper('%исследователь данных%')
            or (name LIKE '%ML%' and name not LIKE '%HTML%')
            or upper(name) like upper('%machine learning%')
            or upper(name) like upper('%машинн%обучен%')
        )
'''
df_16 = pd.read_sql_query(query_16, connection)

In [111]:
df_16

,avg
0,6.406032


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [112]:
query_17 = f'''
select avg((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from)) / 2)
    from public.VACANCIES 
    where (
            upper(name) like upper('%data scientist%') 
            or upper(name) like upper('%data science%')
            or upper(name) like upper('%исследователь данных%')
            or (name LIKE '%ML%' and name not LIKE '%HTML%')
            or upper(name) like upper('%machine learning%')
            or upper(name) like upper('%машинн%обучен%')
        ) and (
            salary_from is not null
            or salary_to is not null
        ) and experience = 'От 3 до 6 лет'
'''
df_17 = pd.read_sql_query(query_17, connection)

In [113]:
df_17

,avg
0,243114.666667


***

# Выводы по предметному анализу

- Предметный анализ показал что только 1% от всех вакансий относится к DS из которых 73% со знаниями Python а это значит что Python остается самый популярный язык в DS
- Средняя зарплата у DS составляет 243 тыс рублей что почти в два раза выше средней от всех вакансий

# Общий вывод по проекту

- Самые крупные IT компанимие предоставляют больше всего вакансий
- Вакансии по DS в среднем выше оплачиваются чем другие сферы